# IBM Data Science Capstone Project
## Coursera First Assignment

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
### Part 1: Webscraping
-----------------------------------------------------------------------------------------------------------------------------------------

In [3]:
# Use bautiful soup to webscrape the web page. 
# - First we need to import adn istall components to use beautiful soup:

# -- !pip install requests
!pip install bs4

# -- Import Libraries
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=b26f1e1c49c1eb7c0a7dd4af074faa580732ea69abb849402e7196cab38f7296
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [4]:
# Create 3 array to capture the requested data for the dataframe
PostalCode = []
Borough = []
Neighborhood = []
NeighTemp = ''

# Wikipedia site URL with the requested information
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Retrieve all information from the website into the data variable
data  = requests.get(url).text 

# create a soup object using the variable 'data'
soup = BeautifulSoup(data,"html5lib")  

In [5]:
# Capture the table content in the 'table' variable.
table = soup.find_all('table')[0]

# Loop throughout the table rows to collect the desired information
for row in table.find_all('td'):
    PostalCode.append(row.contents[1].contents[0].string)
    Borough.append(row.contents[1].contents[2].contents[0].string)
    for i, col in enumerate(row.find_all('a')):
        if i != 0:
            NeighTemp = NeighTemp +  col['title'] + ", "
    Neighborhood.append(NeighTemp)   
    NeighTemp = ''
 

In [6]:
# Create a pandas dataframe with the collected information
df_neighborhoods = pd.DataFrame(list(zip(PostalCode, Borough, Neighborhood)), columns =['PostalCode','Borough','Neighborhood'])

# Check the dataframe
print(df_neighborhoods.shape)
df_neighborhoods.head()

(180, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,"Parkwoods,"
3,M4A,North York,"Victoria Village,"
4,M5A,Downtown Toronto,"Regent Park, Harbourfront, Toronto,"


### Part 2: Data Cleaning
-----------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Remove rows that have Borough as "Not assigned"
df_neighborhoods.drop(df_neighborhoods[df_neighborhoods['Borough'] == 'Not assigned'].index, inplace = True)

# Reset the index values after dropping rows
df_neighborhoods.reset_index(inplace=True)
df_neighborhoods.drop(columns=['index'], axis=0,inplace=True)

# Remove the ", Toronto" values from  the neighbourhoods
df_neighborhoods['Neighborhood'] = df_neighborhoods['Neighborhood'].str.replace(', Toronto, ','')

# Assign Borough value to neighbourhoods woithout value
df_neighborhoods['Neighborhood'].replace('',df_neighborhoods['Borough'],inplace=True)

# Remove commas at the end of the neighbourhoods
for i in range(df_neighborhoods.shape[0]):
    size = len(df_neighborhoods.loc[i]['Neighborhood'])
    if df_neighborhoods.loc[i]['Neighborhood'][size-2:size] == ', ':
        df_neighborhoods.loc[i]['Neighborhood'] = df_neighborhoods.loc[1]['Neighborhood'][:size - 2]

print(df_neighborhoods.shape)
df_neighborhoods.head()

(103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Victoria
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,Victoria Village
4,M7A,Queen's Park,Queen's Park


### Part 3: Get Coordinates
-----------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
!pip install geocoder
import geocoder # import geocoder

In [ ]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('postal_code'),sensor=False)
    lat_lng_coords = g.latlng
    print(g)
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [ ]:
print(lat_lng_coords) 